In [6]:
import os
import swat
import getpass
from sasctl import Session
from sasctl.tasks import register_model, publish_model


Connect to SAS platform (for illustration purposes)

In [7]:
password = getpass.getpass ('Enter your password : ')
hostname = "https://viya4-s2.zeus.sashq-d.openstack.sas.com/"
os.environ["CAS_CLIENT_SSL_CA_LIST"] = r"C:\Users\sbxmip\.vscode\trustedcerts.pem"
conn = swat.CAS('https://viya4-s2.zeus.sashq-d.openstack.sas.com/cas-shared-default-http', username='sbxmip',password=password)
Session(hostname, 'sbxmip', password)


Perform Simple Table Operations

In [8]:

EU_ETS_tbl = conn.CASTable('EU_ETS_CURATED', caslib='EBA')
EU_ETS_tbl.columnInfo()

,Column,Label,ID,Type,RawLength,FormattedLength,Format,NFL,NFD
0,id,Installation id as combination of registry ide...,1,char,288,288,,0,0
1,name,Name of installation,2,char,1024,1024,,0,0
2,tradingSystem_id,Trading system in which the installation is re...,3,char,80,80,,0,0
3,registry_id,2-digit ISO code of registry,4,char,32,32,,0,0
4,activity_id,Key of installation activity,5,double,8,12,,0,0
5,eprtrID,Identifier in the European Pollution Release a...,6,char,304,304,,0,0
6,parentCompany,Name of the parent company of the installation,7,char,512,512,,0,0
7,subsidiaryCompany,Name of subsidiary company of the installation,8,char,1024,1024,,0,0
8,permitID,Permit identifier of the installation,9,char,576,576,,0,0
9,designatorICAO,Aircraft: Designator for the aircraft type acc...,10,char,208,208,,0,0


In [ ]:
EU_ETS_tbl.head(5)

Fit a linear regression model using the GLM Action set

In [9]:
conn.loadActionSet('regression')
conn.regression.glm( 
    table = dict(name = EU_ETS_tbl, where = 'year_comp = 2021'), 
    classVars=['Country_id','nace_desc','activity_desc'],
    model={'depVar':'verified',
                   'effects':['Country_id', 'activity_desc', 'nace_desc']
                  },
    store = dict(name='reg_model', replace=True),
    selection = {"method":"BACKWARD"}
)


NOTE: Added action set 'regression'.
NOTE: 1396306 bytes were written to the table "reg_model" in the caslib "CASUSER(sbxmip)".


,Class,Levels,Values
0,country_id,32.0,AT BE BG CY CZ DE DK EE ES EU FI FR GB GR HR H...
1,nace_desc,283.0,"01.11-Growing of cereals (except rice), legumi..."
2,activity_desc,37.0,1-Combustion installations with a rated therma...
,RowId,Description,Value
0,NEFFECTS,Number of Effects,4
1,NPARMS,Number of Parameters,269
,RowId,Description,Value
0,DATA,Data Source,EU_ETS_CURATED
1,RESPONSEVAR,Response Variable,verified
,RowId,Description,Value


Register and Publish GLM Model

In [11]:
astore = conn.CASTable('reg_model')
model = register_model(astore, 'Glm_2021_back', 'GHG Emissions Model', force=True)



NOTE: Added action set 'astore'.
NOTE: 1396306 bytes were downloaded from the table "REG_MODEL" in the caslib "CASUSER(sbxmip)".


In [ ]:
module = publish_model(model, 'maslocal')

Score first row

In [ ]:
# Pass a row of data to MAS and receive the predicted result.
first_row = EU_ETS_tbl.head(1)
result = module.score(first_row)
print(result)